In [15]:
import pandas as pd
import geopandas as geopd

import numpy as np

import os
import pathlib

from tqdm.notebook import tqdm

In [24]:
# TODO add swe when it is done
order_list = ['discharge_vol', 'discharge_spec', 'precipitation', 'pet', 'pet_era5', 'pet_fmi', 'snow_evaporation', 'swe', 'swe_cci3-1', 'snow_depth', 'temperature_gmin', 'temperature_min', 'temperature_mean', 'temperature_max', 'humidity_rel', 'radiation_global']

calculate_pet = False

#for bilal
#order_list = ['discharge_vol', 'discharge_spec', 'precipitation', 'snow_depth', 'temperature_gmin', 'temperature_min', 'temperature_mean', 'temperature_max', 'humidity_rel', 'radiation_global']

catchments_path = "/home/iielse/Documents/DIWA/data/CAMELS-FI catchments/CAMELS-FI_catchments.gpkg"
src_root = "/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute"
dst_root = "/media/iielse/T9/CAMELS-FI/data/timeseries"
catchments = geopd.read_file(catchments_path, layer='v1')
"""
# for bilal
catchments_path = "/home/iiro/Documents/DIWA/bilal/watersheds.gpkg"
src_root = "/home/iiro/Documents/DIWA/bilal/timeseries_by_attribute"
dst_root = "/home/iiro/Documents/DIWA/bilal/timeseries"
catchments = geopd.read_file(catchments_path, layer='watersheds')
"""
root_path = pathlib.Path(src_root)
# Getting the files in 
files = [item for item in root_path.iterdir() if item.is_file()]

### Combining snow evaporation and era5 pet based on if snow depth > 0

In [26]:
def read_daily_timeseries_csv(path, time_column='date'):
    df = pd.read_csv(path)
    df[time_column] = pd.to_datetime(df[time_column]) 
    df = df.set_index(time_column)
    return df

# Some of the files contain too many catchments (too short flow time series), so the correct ones are read from the definitely correct place.
gauges = list(catchments['Paikka_Id'])

if calculate_pet:

    for file in files:
        attribute = file.stem
        if attribute == 'pet_era5':
            pet_era5 = read_daily_timeseries_csv(file)
    
        if attribute == 'snow_evaporation':
            snow_e = read_daily_timeseries_csv(file)
    
        if attribute == 'snow_depth':
            snow_depth = read_daily_timeseries_csv(file)
            # snow depth has some extra gauges that have been removed
            snow_depth = snow_depth[gauges]
    assert pet_era5.columns.equals(snow_e.columns), "columns don't match"
    assert pet_era5.columns.equals(snow_depth.columns), "columns don't match"
    assert pet_era5.index.equals(snow_e.index), "indices don't match"
    assert pet_era5.index.equals(snow_depth.index), "indices don't match"
    
    pet = pd.DataFrame(np.where(snow_depth > 0, snow_e, pet_era5), index=pet_era5.index, columns=pet_era5.columns)
    
    pet_path = os.path.join(src_root, 'pet.csv')
    pet_path = pathlib.Path(pet_path)
    pet.to_csv(pet_path)
    if pet_path not in files:
        files.append(pet_path)
    

In [27]:
files

[PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/precipitation.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/temperature_min.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/swe_cci3-1.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/pet_era5.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/temperature_mean.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/temperature_max.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/radiation_global.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/snow_depth.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/temperature_gmin.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/swe.csv'),
 PosixPath('/media/iielse/T9/CAMELS-FI/data/timeseries_by_attribute/pet_fmi.csv'),
 PosixPath('/media/iielse/T9/CAMELS

In [21]:
pet

,996,1003,1008,1009,1012,1014,1015,1018,1019,1021,...,937,941,943,946,951,961,964,1369,1366,1357
date,,,,,,,,,,,,,,,,,,,,,
1961-01-01,0.02,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.02,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.09,0.09,0.10
1961-01-02,0.00,0.03,0.04,0.03,0.03,0.03,0.02,0.02,0.00,-0.01,...,0.06,0.07,0.07,0.07,0.06,0.06,0.05,0.06,0.07,0.02
1961-01-03,0.09,0.07,0.07,0.07,0.08,0.09,0.08,0.08,0.09,0.11,...,0.09,0.10,0.10,0.09,0.09,0.08,0.08,0.04,0.04,0.05
1961-01-04,0.19,0.15,0.15,0.15,0.15,0.17,0.17,0.17,0.18,0.19,...,0.16,0.14,0.14,0.14,0.13,0.12,0.13,0.22,0.23,0.14
1961-01-05,0.05,0.04,0.04,0.04,0.04,0.03,0.04,0.04,0.03,0.03,...,0.06,0.07,0.07,0.06,0.05,0.02,0.03,0.10,0.09,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,-0.01,-0.02,-0.02,-0.02,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,...,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,-0.01,-0.01,-0.01,-0.01
2023-12-28,-0.02,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,...,0.00,-0.00,-0.01,-0.01,-0.01,-0.02,-0.02,-0.02,-0.01,-0.01
2023-12-29,0.09,0.02,0.02,0.02,0.03,0.05,0.05,0.05,0.08,0.12,...,0.00,-0.01,-0.01,-0.01,-0.01,-0.01,0.01,-0.01,-0.01,-0.01


In [28]:
for gauge in tqdm(gauges):
    columns = []
    
    for file in files:
        attribute = file.stem
        meteo = read_daily_timeseries_csv(file)

        #column = meteo[[gauge]]
        
        
        column = meteo.loc["1961":"2023", [gauge]]
        column = column.rename({gauge: attribute}, axis=1)
        columns.append(column)
    output = pd.concat(columns, axis=1)
    # reordering the columns
    output = output[order_list]
    dst_path = os.path.join(dst_root, f"CAMELS_FI_hydromet_timeseries_{gauge}_19610101-20231231.csv")
    output.to_csv(dst_path)
    break

  0%|          | 0/320 [00:00<?, ?it/s]

In [17]:
str(meteo.index[0].date())

'1961-01-01'

In [19]:
meteo

,996,1003,1008,1009,1012,1014,1015,1018,1019,1021,...,937,941,943,946,951,961,964,1369,1366,1357
date,,,,,,,,,,,,,,,,,,,,,
1961-01-01,6.0,3.7,3.6,3.6,3.6,3.3,4.5,4.5,4.6,3.2,...,3.0,3.1,3.1,3.4,3.8,5.1,3.2,2.3,2.4,0.9
1961-01-02,2.9,2.3,1.9,1.9,1.9,1.7,2.4,2.4,2.2,2.4,...,2.1,1.4,1.4,1.5,1.3,1.0,1.1,3.5,3.3,2.8
1961-01-03,0.8,1.0,0.9,0.9,0.9,1.0,0.8,0.8,0.6,0.1,...,1.5,1.3,1.3,1.2,1.0,0.2,0.9,1.4,1.4,0.6
1961-01-04,5.4,4.0,4.5,4.5,4.7,5.3,4.7,4.8,7.3,8.3,...,2.6,3.0,3.1,3.5,3.8,4.3,4.6,0.9,1.1,0.7
1961-01-05,2.0,2.1,2.6,2.6,2.6,3.3,2.2,2.2,2.1,1.4,...,2.4,2.1,2.2,2.2,2.2,2.1,3.5,4.6,4.7,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,0.1,0.7,0.6,0.6,0.6,0.3,0.4,0.3,0.0,0.0,...,0.4,0.3,0.3,0.4,0.5,0.5,0.5,0.0,0.0,0.0
2023-12-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.1,0.1,0.0,0.1,0.0,0.0,0.1,0.1,0.0
2023-12-29,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.2
